In [1]:
import pandas as pd
from taskB_ml_baseline_utils import FeatureMaker, calculate_idfs

In [3]:
df = pd.DataFrame.from_csv(
    '../../train.csv', sep=',', index_col=None)
df = df[['paragraph_id', 'question_id', 'paragraph', 'question', 'answer']]

In [13]:
df.head()

,paragraph_id,question_id,paragraph,question,answer
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные


Считаем IDF слов

In [4]:
idfs = calculate_idfs(df)

calc idf: 100%|██████████| 9078/9078 [00:00<00:00, 11881.28it/s]


Строим фичи объектов

In [5]:
maker = FeatureMaker(df.head(10000), idfs)
train = maker.make()

calc features: 100%|██████████| 10000/10000 [01:42<00:00, 97.72it/s]


In [6]:
train.columns

Index(['ADJF', 'ADJS', 'ADVB', 'COMP', 'CONJ', 'GRND', 'IDF (Span)', 'INFN',
       'INTJ', 'LATN', 'Left Length', 'Match IDF  (Span)',
       'Match IDF (Right of Span)', 'Match IDF (Whole Sentence)',
       'Match TF IDF (Left of Span)', 'NOUN', 'NPRO', 'NUMR', 'PRCL', 'PRED',
       'PREP', 'PRTF', 'PRTS', 'ROMN', 'Right Length', 'Sentence Length',
       'Span Length', 'UNKN', 'VERB', 'question_id', 'target'],
      dtype='object')

Обучаем модель

In [7]:
from sklearn.ensemble import GradientBoostingRegressor as GBR
model = GBR()
model.fit(train[train.columns.difference(['question_id', 'target', 'paragraph_id'])], train['target'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

Сохраняем модель

In [11]:
from sklearn.externals import joblib
joblib.dump(model, 'model.pkl');

Для применения модели нам также надо знать IDF слов, поэтому сохраняем и их тоже

In [12]:
import pickle
with open("idfs.pkl", "wb") as f:
    pickle.dump(idfs, f)